### Importazione librerie e caricamento dataset

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from scipy.stats import pearsonr, spearmanr, chi2_contingency
from sklearn.preprocessing import LabelEncoder
import numpy as np

In [ ]:
#MODIFICARE CON DATASET CON IMPUTATIONS

df = pd.read_csv('../dataset/cyclists.csv')

# Separa le variabili numeriche e categoriche
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns
categorical_features = df.select_dtypes(include=['object', 'category']).columns

# One-Hot Encoding per variabili categoriche nominali
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)
df.head()

In [ ]:
# Separazione delle variabili numeriche e categoriche
numerical_features = df.select_dtypes(include=['float64', 'int64']).columns
categorical_features = df.select_dtypes(include=['object', 'category']).columns
print("Numerical Features:", numerical_features)
print("Categorical Features:", categorical_features)

### Informazioni statistiche di base

In [ ]:
df.describe()

### Distribuzione variabili numeriche

In [ ]:
for col in numerical_features:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[col], kde=True, bins=30)
    plt.title(f'Distribuzione di {col}')
    plt.xlabel(col)
    plt.ylabel('Frequenza')
    plt.show()

for col in numerical_features:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=df[col])
    plt.title(f'Boxplot della Variabile Numerica: {col}')
    plt.xlabel(col)
    plt.show()

### Distribuzione variabili categoriche

In [ ]:
for cat in categorical_features:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=df[cat])
    plt.title(f'Frequenza delle Categorie per {cat}')
    plt.xlabel(cat)
    plt.ylabel('Conteggio')
    plt.xticks(rotation=45)
    plt.show()

# Boxplot e Violin Plot per variabili numeriche suddivise per categorie
for cat in categorical_features:
    if df[cat].nunique() <= 10:  # Limita ai grafici solo le variabili categoriche con meno di 10 categorie
        for num in numerical_features:
            plt.figure(figsize=(8, 6))
            sns.boxplot(x=df[cat], y=df[num])
            plt.title(f'Distribuzione di {num} per i livelli di {cat} (Boxplot)')
            plt.show()

            plt.figure(figsize=(8, 6))
            sns.violinplot(x=df[cat], y=df[num])
            plt.title(f'Distribuzione di {num} per i livelli di {cat} (Violin Plot)')
            plt.show()


### Normalizzazione

In [ ]:
scaler = MinMaxScaler()
numeric_columns = df.select_dtypes(include='number').columns
normalized_df = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns)
print("DataFrame Normalizzato:")
print(normalized_df)

### Nuove distribuzioni

In [ ]:
numeric_columns = normalized_df.select_dtypes(include='number').columns
for col in numeric_columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(normalized_df[col], kde=True, bins=30)
    plt.title(f'Distribuzione di {col}')
    plt.show()

### Standardizzazione

In [ ]:
scaler = StandardScaler()
numeric_columns = df.select_dtypes(include='number').columns
standardized_df = pd.DataFrame(scaler.fit_transform(df[numeric_columns]), columns=numeric_columns)
print("\nDataFrame Standardizzato:")
print(standardized_df)


### Nuove distribuzioni

In [ ]:
numeric_columns = standardized_df.select_dtypes(include='number').columns
for col in numeric_columns:
    plt.figure(figsize=(8, 6))
    sns.histplot(standardized_df[col], kde=True, bins=30)
    plt.title(f'Distribuzione di {col}')
    plt.show()

### Correlazioni tra variabili numeriche

In [ ]:
methods = ['pearson', 'spearman', 'kendall']
for method in methods:
    correlation_matrix = df[numerical_features].corr(method=method)
    plt.figure(figsize=(12, 8))
    sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', linewidths=0.5, fmt='.2f')
    plt.title(f'Heatmap delle correlazioni ({method.capitalize()})')
    plt.show()

### Correlazioni tra variabili categoriche

In [ ]:
# Funzione per calcolare il Cramér's V
def cramers_v(x, y):
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    return np.sqrt(chi2 / (n * (min(r, k) - 1)))

# Calcola la correlazione tra tutte le coppie di variabili categoriche
categorical_corr = pd.DataFrame(index=categorical_features, columns=categorical_features)
for col1 in categorical_features:
    for col2 in categorical_features:
        if col1 == col2:
            categorical_corr.loc[col1, col2] = 1
        else:
            categorical_corr.loc[col1, col2] = cramers_v(df[col1], df[col2])

# Visualizza la matrice di correlazione categorica
plt.figure(figsize=(10, 8))
sns.heatmap(categorical_corr.astype(float), annot=True, cmap='coolwarm')
plt.title('Matrice di Correlazione per Variabili Categoriali (Cramér\'s V)')
plt.show()


### Pair plot (relazioni tra variabili numeriche)

In [ ]:
sns.pairplot(df[numerical_features])
plt.show()

In [ ]:
# Analisi delle distribuzioni

# Istogrammi per le variabili numeriche
for num in numerical_features:
    plt.figure(figsize=(8, 6))
    sns.histplot(df[num].dropna(), kde=True)
    plt.title(f'Distribuzione della Variabile Numerica: {num}')
    plt.xlabel(num)
    plt.ylabel('Frequenza')
    plt.show()

# Boxplot per le variabili numeriche
for num in numerical_features:
    plt.figure(figsize=(8, 6))
    sns.boxplot(x=df[num])
    plt.title(f'Boxplot della Variabile Numerica: {num}')
    plt.xlabel(num)
    plt.show()

# Grafici a barre per le variabili categoriche
for cat in categorical_features:
    plt.figure(figsize=(8, 6))
    sns.countplot(x=df[cat])
    plt.title(f'Frequenza delle Categorie per {cat}')
    plt.xlabel(cat)
    plt.ylabel('Conteggio')
    plt.xticks(rotation=45)
    plt.show()

In [ ]:
# Standardizzazione e Normalizzazione delle Variabili Numeriche

# Standardizzazione (media 0, deviazione standard 1)
scaler = StandardScaler()
df[numerical_features] = scaler.fit_transform(df[numerical_features])

# Normalizzazione (range [0,1]) per confronto
normalizer = MinMaxScaler()
df_normalized = df.copy()
df_normalized[numerical_features] = normalizer.fit_transform(df[numerical_features])

df.head()  # Visualizza i dati trasformati

In [ ]:
# Analisi della Correlazione Completa con Variabili Codificate

# Esegue One-Hot Encoding per le variabili categoriche nominali
df_encoded = pd.get_dummies(df, columns=categorical_features, drop_first=True)

# Calcola la matrice di correlazione completa
encoded_features = df_encoded.select_dtypes(include=['float64', 'int64']).columns
corr_matrix = df_encoded[encoded_features].corr()

# Visualizza la matrice di correlazione
plt.figure(figsize=(12, 10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title('Matrice di Correlazione Completa con Variabili Codificate')
plt.show()